In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
res = client.models.generate_content(model=MODEL_ID,contents='Quando é a próxima imersão da alura?')
res.text

'A Alura geralmente realiza imersões com frequência, mas não tenho informações sobre uma data específica para a próxima. A melhor forma de saber quando será a próxima imersão é:\n\n*   **Acessar o site da Alura:** Procure na página inicial ou na seção de eventos.\n*   **Seguir as redes sociais da Alura:** Eles costumam anunciar as imersões no Instagram, LinkedIn, Twitter e outras plataformas.\n*   **Assinar a newsletter da Alura:** Assim, você receberá informações sobre os próximos eventos diretamente no seu e-mail.\n*   **Verificar o fórum da Alura:** A comunidade pode ter informações sobre próximas imersões.'

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {res.text}"))

Resposta:
 A Alura geralmente realiza imersões com frequência, mas não tenho informações sobre uma data específica para a próxima. A melhor forma de saber quando será a próxima imersão é:

*   **Acessar o site da Alura:** Procure na página inicial ou na seção de eventos.
*   **Seguir as redes sociais da Alura:** Eles costumam anunciar as imersões no Instagram, LinkedIn, Twitter e outras plataformas.
*   **Assinar a newsletter da Alura:** Assim, você receberá informações sobre os próximos eventos diretamente no seu e-mail.
*   **Verificar o fórum da Alura:** A comunidade pode ter informações sobre próximas imersões.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config = {"tools":[{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 As inscrições para a Imersão IA da Alura + Google Gemini vão até o dia 11 de maio de 2025. A Imersão IA é um curso online e gratuito que visa impulsionar sua carreira com IA na prática, ensinando a criar prompts, automatizar tarefas e desenvolver sistemas de agentes autônomos. Ao final da Imersão, você pode ganhar um certificado Alura + Google e concorrer a prêmios com seu projeto.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura próxima imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agenda de IA
Maravilha! Vamos então criar o post sobre novidades em agenda de IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Aqui estão algumas das notícias mais recentes sobre lançamentos relacionados à agenda de IA, com foco em anúncios recentes e significativos:
> 
> 
> Com base nas notícias mais recentes, aqui estão alguns lançamentos e tendências relevantes na área de IA:
> 
> 1.  **Modelos de IA GPT-4.1 e GPT-4.1 Mini da OpenAI:** A OpenAI lançou os novos modelos de IA GPT-4.1 e GPT-4.1 Mini para usuários do ChatGPT. O GPT-4.1 promete um desempenho mais rápido e melhorias significativas em tarefas de programação, enquanto o modelo mini estará acessível a todos os usuários da plataforma.
> 
> 2.  **Ferramentas de IA para diversas aplicações:** Há um crescente número de ferramentas de IA sendo lançadas para transformar a forma como abordamos diversas tarefas. Estas ferramentas incluem Agent.ai, uma rede profissional para agentes de inteligência artificial, e ferramentas para automação de tarefas, análise de dados e geração de conteúdo.
> 
> 3.  **IA para Automação Conversacional:** A automação conversacional, com chatbots, assistentes virtuais e agentes inteligentes, tem ganhado destaque. As ferramentas de IA estão se tornando multimodais, combinando texto, voz, imagem e vídeo para interações mais naturais.
> 
> 4.  **Jornada de IA 2025 da ANBIMA:** A ANBIMA está promovendo a "Jornada de Inteligência Artificial 2025", uma série de eventos focados em temas como governança, segurança cibernética e regulação da IA no mercado de capitais.
> 
> 5.  **IA na Comunicação Pública:** O programa IntegraCom da USP está promovendo um evento para discutir os desafios éticos e regulatórios da IA na comunicação pública. A discussão visa alinhar o uso de ferramentas de IA aos princípios éticos do jornalismo.
> 
> É importante notar que a área de IA está em rápida evolução, com novos modelos e ferramentas sendo lançados constantemente. Além disso, há um crescente interesse em garantir que a IA seja usada de forma ética e responsável.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos e tendências de IA que você me forneceu, vou criar um plano de conteúdo para um post de mídia social sobre o tema mais relevante. Para isso, farei algumas pesquisas adicionais para aprofundar meu conhecimento sobre os tópicos.
> 
> 
> Após analisar as informações disponíveis, o tema mais relevante para um post de mídia social parece ser a **Jornada de IA 2025 da ANBIMA** e os **desafios éticos e regulatórios da IA na comunicação pública**, dada a sua atualidade e importância para o mercado financeiro e para a sociedade em geral.
> 
> **Tema:** Jornada de IA 2025 da ANBIMA e Desafios Éticos na Comunicação Pública.
> 
> **Pontos Relevantes:**
> 
> *   **Implementação Prática da IA:** A Jornada da ANBIMA foca na implementação prática da IA no mercado de capitais, o que é crucial para as instituições financeiras.
> *   **Governança e Segurança:** A série de eventos aborda temas como governança, segurança cibernética e regulação, que são preocupações chave na adoção da IA.
> *   **Desafios Éticos e Regulatórios:** O evento IntegraCom da USP destaca a necessidade de alinhar o uso da IA com os princípios éticos, especialmente no jornalismo e na comunicação pública.
> *   **Cibersegurança:** A Jornada de IA 2025 da ANBIMA também está discutindo os novos desafios de cibersegurança com a ampliação das possibilidades de ataques e ameaças mais sofisticadas com a IA.
> 
> **Plano para o Post:**
> 
> 1.  **Introdução:**
>     *   Começar com uma pergunta chamativa sobre o futuro da IA no Brasil.
>     *   Mencionar que a IA está transformando diversos setores, incluindo o mercado financeiro e a comunicação pública.
> 2.  **Jornada de IA 2025 da ANBIMA:**
>     *   Apresentar a Jornada como uma iniciativa importante para o mercado de capitais.
>     *   Destacar os temas principais: governança, segurança cibernética, infraestrutura, regulação e compliance.
>     *   Incentivar o público a se inscrever nos eventos online.
> 3.  **Desafios Éticos na Comunicação Pública:**
>     *   Abordar a importância de usar a IA de forma ética e responsável na comunicação.
>     *   Mencionar o evento IntegraCom da USP e a discussão sobre os desafios éticos e regulatórios.
>     *   Discutir a necessidade de transparência, responsabilidade e veracidade no uso da IA no jornalismo.
> 4.  **Impacto e Oportunidades:**
>     *   Explorar como a IA pode melhorar a eficiência e a inovação no mercado financeiro.
>     *   Discutir o potencial da IA para transformar a comunicação pública e o jornalismo.
>     *   Enfatizar a importância de equilibrar os benefícios da IA com a necessidade de mitigar os riscos éticos e de segurança.
> 5.  **Chamada para Ação:**
>     *   Incentivar o público a compartilhar suas opiniões sobre o futuro da IA nos comentários.
>     *   Convidar as pessoas a se inscreverem nos eventos da ANBIMA e da USP para se manterem informadas.
> 
> Este plano busca fornecer uma visão geral dos principais lançamentos e tendências na área de IA, com foco em como a IA está sendo implementada e regulamentada no Brasil.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## Rascunho de Post para Instagram:
> 
> **Imagem:** Uma imagem impactante que combine elementos de tecnologia, finanças e ética, como um cérebro digital com circuitos integrados e um símbolo de balança.
> 
> **Texto:**
> 
> E aí, futuro tá on? 🚀 A Inteligência Artificial já chegou pra ficar e tá transformando tudo, desde o mercado financeiro até a forma como nos comunicamos! 🤖
> 
> A **Jornada de IA 2025 da ANBIMA** tá rolando e é essencial pra quem quer entender como a IA vai revolucionar o mercado de capitais. 📈 Eles estão debatendo temas como governança, segurança cibernética e regulamentação. Fica ligado nos eventos online! 😉
> 
> Mas não é só isso! 🤔 O uso da IA na comunicação pública também levanta questões importantes. Como garantir que a IA seja usada de forma ética e responsável no jornalismo? O evento IntegraCom da USP está discutindo exatamente isso! 📣
> 
> A IA tem um potencial incrível pra trazer inovação e eficiência, mas também precisamos ficar de olho nos desafios éticos e de segurança. 🔒
> 
> E você, o que acha do futuro da IA no Brasil? Compartilhe sua opinião nos comentários! 👇 E não se esqueça de se inscrever nos eventos da ANBIMA e da USP pra ficar por dentro de tudo! 😉
> 
> #IA #InteligenciaArtificial #ANBIMA #EticaNaIA
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais atrativo para o público do Instagram, especialmente a faixa etária de 18 a 30 anos. Aqui estão algumas sugestões:
> 
> *   **Tom e Linguagem:** Podemos usar uma linguagem um pouco mais informal e direta, com mais emojis para criar conexão.
> 
> *   **Chamada para Ação (CTA):** Tornar a chamada para ação mais direta e envolvente.
> 
> **Sugestões de Melhorias:**
> 
> *   **Imagem:** Perfeito! A imagem que combina tecnologia, finanças e ética é bem chamativa.
> *   **Texto:**
> 
> E aí, futuristas! 🚀 A Inteligência Artificial já invadiu TUDO e tá mudando o game em áreas que você nem imagina, tipo no mercado financeiro e até no jeito que a gente troca ideia! 🤖🤯
> 
> A **Jornada de IA 2025 da ANBIMA** tá rolando e é O LUGAR pra entender como a IA vai dar um UPGRADE no mercado de investimentos. 📈 Eles estão debatendo temas super importantes como segurança online e como as regras do jogo tão mudando. Se liga nos eventos online que estão rolando! 😉
> 
> Mas não para por aí! 🙅‍♀️ A IA também tá sendo usada na comunicação e isso levanta umas questões... Será que a IA está sendo usada de forma honesta e responsável no jornalismo? 🤔 O evento IntegraCom da USP está DEBATE isso! 📣
> 
> A IA tem um poder INCRÍVEL pra trazer coisas novas e fazer tudo funcionar melhor, mas também temos que ficar espertos com os problemas que ela pode trazer. 🔒
> 
> E aí, qual a sua opinião sobre o futuro da IA aqui no Brasil? Solta o verbo nos comentários! 👇 E não esquece de se inscrever nos eventos da ANBIMA e da USP pra não perder NADA! Link na bio! 😉
> 
> *   **Hashtags:** As hashtags estão boas, mas podemos adicionar algumas mais específicas e populares no momento para aumentar o alcance.
> 
> **Hashtags Sugeridas:**
> 
> `#IA #InteligenciaArtificial #ANBIMA #EticaNaIA #FuturoDaTecnologia #Inovacao #MercadoFinanceiro #OpenAI #Chatbots #TecnologiaBrasil`
> 
> Com essas pequenas mudanças, o post ficará ainda mais atraente e engajador para o público do Instagram!


--------------------------------------------------------------
